In [51]:
import os
from collections import defaultdict, Counter
from emailtome import noticeEMail
import urllib, json, csv
import emailtome
import datetime

In [52]:
starttime=datetime.datetime.now()

In [53]:
"""def read_aggr_file(p, directory, filename, et): #aggregate the data and generates user based json files
    pres=[]
    with open(os.path.join(directory,filename), 'r') as myfile:
        data=myfile.read()
    #print 'opened the file'
    lines=data.splitlines()
    for i in range(0, 1000):
        event=json.loads(lines[i].strip())
        #print event.keys()
        #types.add(event['type'])
        if event['type']==et:
            PRE=event
            return PRE"""

"def read_aggr_file(p, directory, filename, et): #aggregate the data and generates user based json files\n    pres=[]\n    with open(os.path.join(directory,filename), 'r') as myfile:\n        data=myfile.read()\n    #print 'opened the file'\n    lines=data.splitlines()\n    for i in range(0, 1000):\n        event=json.loads(lines[i].strip())\n        #print event.keys()\n        #types.add(event['type'])\n        if event['type']==et:\n            PRE=event\n            return PRE"

In [56]:
def date_from_filename(fname):
    dt = datetime.datetime.strptime(fname[len('githubarchive_'):-len('.json')], '%Y-%m-%d')
    return dt.strftime('%Y-%m')

In [57]:
def date_from_json(creation_date):
    dt = datetime.datetime.strptime(creation_date[:10], '%Y-%m-%d')
    return dt.strftime('%Y-%m')

In [58]:
def url_to_json(link):
    url = link
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    return data

In [59]:
def parse(event, date_f, merging_matter):
    #print event['type']
    if type(event)==dict:
        if 'repo' in event.keys(): 
            #print'k'
            if 'id' in event['repo'].keys():
                #print 'id in repo'
                if merging_matter==False:
                    d=date_f
                    if 'user' in event.keys():
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                    elif 'actor' in event.keys():
                        if 'login' in event['actor'].keys():
                            u=event['actor']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                        elif 'payload' in event.keys():
                            if 'actor' in event['payload'].keys():
                                u=event['payload']['actor']
                                p=event['repo']['id']
                                return (u, p, d)
                elif merging_matter==True:
                    d=date_from_json(event['payload']['pull_request']['created_at'][:10])
                    #print d
                    if 'user' in event.keys():
                        #print event.keys()
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)      
                    elif 'actor' in event.keys():
                        u=event['actor']['login']
                        p=event['repo']['id']
                        return (u, p, d)
            else:
                return ('error' ,_,_)
        elif 'repository' in event.keys():
            u=event['actor']
            p=event['repository']['id']
            d=date_f
            return (u, p, d)
        else:
            return ('error' ,_,_)
        

In [60]:
#returns user, project, date, number of commits pushed
def parse_number_of_pushed_commits(event, date_f):
    u,p,d=parse_push(event, date_f)
    c=len(event['payload']['commits'])
    return(u,p,d,c)

In [61]:
def parse_follow_event(event):
    if type(event['actor'])==dict:
        f=event['payload']['actor']
        t=event['payload']['target']['login']
    else:
        f=event['actor']
        t=event['payload']['target']['login']
    return (f,t)

In [62]:
def parse_membership_event(event):
    u=event['actor']['login']
    o=event['org']['login']
    a=event['payload']['action']
    return (u,o,a) 

In [63]:
def parse_member_event(event):
    if 'repo' not in event.keys():
        u=event['actor']
        r=event['repository']['name']
        o=event['repository']['owner']
        a=event['payload']['action']
        return (u,r,o,a) 
    elif type(event['payload']['member'])==dict:
        u=event['payload']['member']['login']
        r=event['repo']['name']
        o=event['actor']['login']
        a=event['payload']['action']
    else:
        u=event['payload']['member']
        r=event['repo']['name']
        o=event['payload']['actor']
        a=event['payload']['action']
        return (u,r,o,a)         

In [64]:
def get_projects_dict(all_projects, user, project, date, eventt):
    if project not in all_projects.keys():
        events=defaultdict(dict)
        users=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        users[user]=dates
        events[eventt]=users
        all_projects[project]=events
        return all_projects
    elif eventt not in all_projects[project].keys():
        users=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        users[user]=dates
        all_projects[project][eventt]=users
        return all_projects 
    elif user not in all_projects[project][eventt].keys():
        dates=defaultdict(dict)
        dates[date]=1
        all_projects[project][eventt][user]=dates
        #print projects[project].keys()
        return all_projects
    elif date not in all_projects[project][eventt][user].keys():
        dates=defaultdict(dict)
        dates[date]=1
        all_projects[project][eventt][user]=dates
        return all_projects
    else:
        all_projects[project][eventt][user][date]+=1
        return all_projects

In [65]:
def get_user_activity(user_activity, user, project, date, eventt):
    if user not in user_activity.keys():
        projects=defaultdict(dict)
        events=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        projects[project]=events
        user_activity[user]=projects
        return user_activity
    elif project not in user_activity[user].keys():
        projects=defaultdict(dict)
        events=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        projects[project]=events
        user_activity[user][project]=events
        return user_activity 
    elif eventt not in user_activity[user][project].keys():
        dates=defaultdict(dict)
        events=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        user_activity[user][project][eventt]=dates
        return user_activity
    elif date not in user_activity[user][project][eventt].keys():
        dates=defaultdict(dict)
        dates[date]=1
        user_activity[user][project][eventt]=dates
        return user_activity
    else:
        user_activity[user][project][eventt][date]+=1
        return user_activity

In [66]:
#TEST
"""all_projects=defaultdict(dict)
p_u=((1,30, datetime.date(2015, 12, 29), 'pd'), (2,30, datetime.date(2015, 12, 30), 'pd'), (1,20, datetime.date(2015, 12, 29), 'ab'), (1,20, datetime.date(2015, 12, 29), 'ab'))
for (u, p, d, e) in p_u:
    h=get_projects_dict(all_projects, u, p, d, e)"""

"all_projects=defaultdict(dict)\np_u=((1,30, datetime.date(2015, 12, 29), 'pd'), (2,30, datetime.date(2015, 12, 30), 'pd'), (1,20, datetime.date(2015, 12, 29), 'ab'), (1,20, datetime.date(2015, 12, 29), 'ab'))\nfor (u, p, d, e) in p_u:\n    h=get_projects_dict(all_projects, u, p, d, e)"

In [67]:
def all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity):
    et=event['type']
    if et=='FollowEvent':
        f,t=parse_follow_event(event)
        follow_follower_network.append((f,t,date_file))
        users.add(f)
        users.add(t)
    elif et=='MembershipEvent':
        u,o,a=parse_membership_event(event)
        memberships.append((u,o,a, date_file))
        users.add(u)
    elif et=='MemberEvent':
        u,r,o,a=parse_member_event(event)
        repo_memberships.append((u,r,o,a,date_file))
        users.add(u)
        users.add(o)
    elif et=='PullRequestEvent' and event['payload']['action']=='closed':
        if 'merged' in event ['payload']['pull_request']:
            if event['payload']['pull_request']['merged']==True:
                user, project, date=parse(event, date_file, True)
                all_projects=get_projects_dict(all_projects, user, project, date, u'Closed_PullRequestEvent')
                user_activitiy=get_user_activity(user_activity, user, project, date, et)
                #return all_projects, follow_follower_network, memberships, repo_memberships, users
    elif et not in ['GistEvent','GollumEvent','CreateEvent','DeleteEvent','MemberEvent']:
        user, project, date=parse(event, date_file, False)
        if user!='error':
            all_projects=get_projects_dict(all_projects, user, project, date, et)
            user_activity=get_user_activity(user_activity, user, project, date, et)
    return all_projects, follow_follower_network, memberships, repo_memberships, users,user_activity

In [68]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

In [69]:
def write_json(p, d, file_name, data):
        with open(p+str(d)+file_name+".json","w") as f:
            json.dump(data,f, cls=SetEncoder)

In [70]:
def write_csv(p, d, file_name, data):
    with open(p+str(d)+file_name+".csv","w") as csvfile:
        a=csv.writer(csvfile, delimiter=',')
        a.writerows(data)

In [71]:
def write_users(p, d, file_name, data):
    with open(p+str(d)+file_name+".csv","w") as u:
        u.write( '\n'.join(users) )

In [72]:
def write_all(path, date_file, all_projects, follow_follower_network, memberships, repo_memberships, user_activity, users):         
    write_json(path, date_file, '_all_projects', all_projects)
    write_csv(path, date_file, '_follow_follower_network', follow_follower_network)
    write_csv(path, date_file, '_memberships', memberships)
    write_csv(path, date_file, '_repo_memberships', repo_memberships)
    write_json(path, date_file, '_user_activity', user_activity)
    write_users(path, date_file, '_users', users)

In [73]:
def read_lines(data, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity):
    lines=data.splitlines()
    for i in range(0,len(lines)):
        last_line=lines[i]
        try:
            event=json.loads(unicode(lines[i], errors='ignore').strip())
            last_event=event
            all_projects,follow_follower_network, memberships, repo_memberships,users, user_activity=all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)
        except ValueError:
            for k in ['{"repo"'+line for line in lines[i].split('{"repo"')[1:]]:
                event=json.loads(k)
                all_projects,follow_follower_network, memberships, repo_memberships, users, user_activity=all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)
    return all_projects,follow_follower_network, memberships, repo_memberships, users, user_activity

In [76]:
path=''
directory=''

In [78]:
"""projects={project_id: user_id{pull_requests_merged: {date: count},
                        pull_requests_non_merged: {date: count},
                        pushes: {date: count},
                        forks: {date: count},
                        watches: {date: count},
                        
user_activity={user_id: project_id{pull_requests_merged: {date: count},
                        pull_requests_non_merged: {date: count},
                        pushes: {date: count},
                        forks: {date: count},
                        watches: {date: count},
                        stars: {date: count}"""


'projects={project_id: user_id{pull_requests_merged: {date: count},\n                        pull_requests_non_merged: {date: count},\n                        pushes: {date: count},\n                        forks: {date: count},\n                        watches: {date: count},\n                        \nuser_activity={user_id: project_id{pull_requests_merged: {date: count},\n                        pull_requests_non_merged: {date: count},\n                        pushes: {date: count},\n                        forks: {date: count},\n                        watches: {date: count},\n                        stars: {date: count}'

In [ ]:
all_projects=defaultdict(dict)
user_activity=defaultdict(dict)
follow_follower_network=[]
memberships=[]
repo_memberships=[]
users=set()
directory=''
path=os.getcwd()
count=0
files=sorted([i for i in os.listdir(path) if i.startswith("githubarchive") and i.endswith('json')])
number_of_elements=len(files)
old_date=date_from_filename(files[0])
for f in files:
    print f
    count+=1
    with open(os.path.join(path,f), 'r') as myfile:
        data=myfile.read()
        date_file=date_from_filename(f)
        if date_file==old_date:
            print date_file, old_date
            old_date=date_file
            all_projects,follow_follower_network, memberships, repo_memberships, users, user_activity=read_lines(data, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)
            if count==number_of_elements:
                print date_file, old_date, count, number_of_elements
                write_all(directory, date_file, all_projects, follow_follower_network, memberships, repo_memberships, user_activity, users)
        else:
            if count==number_of_elements:
                print date_file, old_date, count, number_of_elements
                write_all(directory, date_file, all_projects, follow_follower_network, memberships, repo_memberships, user_activity, users)
            else:
                print date_file, old_date, count, number_of_elements
                write_all(directory, date_file, all_projects, follow_follower_network, memberships, repo_memberships, user_activity, users)
                all_projects=defaultdict(dict)
                user_activity=defaultdict(dict)
                follow_follower_network=[]
                memberships=[]
                repo_memberships=[]
                users=set()
                all_projects,follow_follower_network, memberships, repo_memberships, users, user_activity=read_lines(data, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)

In [ ]:
usr=''
psw=''
fromaddr=''
toaddr=''
emailtome.noticeEMail(starttime, usr, psw, fromaddr, toaddr)